# Observations and Insights

<hr>

In [1]:
# Dependencies and Setup (John)
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
import calendar

# CalFresh data file path (DeJuan)
calfresh_data_path = "Resources/CalFresh Data 19 - Current.csv"
# Census data file path (Jackson)
census_data_path = "Resources/co-est2019-alldata.csv"
# us-counties data file path (Siddharth)
all_counties_path = "Resources/us-counties.csv"


In [2]:
# Read the all three datasets (John)
calfresh_data = pd.read_csv(calfresh_data_path)

# rename columns.
calfresh_data.rename(columns = {'Calendar Year' : 'Year', 'Unemployment Monthly	' : 'Unemployment', 
                                'CalFresh Households' : 'Households', 'CalFresh Persons' : 'Persons', 
                                'EBT_FSP_dollars' : 'EBT Cash'}, inplace = True)

# removed 'del calfresh_data['Date]'.  
# Use .drop function. Removed columns 7-8 as not needed, and index 0 (row 1). (John)
calfresh_data.drop(calfresh_data.columns[[2, 7, 8]], axis = 1, inplace = True)

##****## Ask TA why index 0 can't be dropped
calfresh_data.drop(labels=[0], axis=0)
#calfresh_data.drop(calfresh_data.index [[ 0]])
##****##

# remove commas in the entire dataframe and convert number strings to a number int. (John)
calfresh_df = pd.DataFrame(calfresh_data)
calfresh_df.replace(',','', regex=True, inplace=True)
# dictionary to convert specific columns to number dtype (John)

calfresh_df = calfresh_df.astype({'Households': 'int64', 'Persons': 'int64', 'EBT Cash': 'int64'})

#Filter calfreash data by 2020 and 2021 (DHall)
calfresh_data20_21 = calfresh_df[calfresh_df['Year']!=2019]

#Export Updated CSV to Resource Folder (DHall)
calfresh_data20_21.to_csv('Resources/calfresh_data20_21.csv')

calfresh_data20_21

,County,Month,Year,Unemployment Monthly,Households,Persons,EBT Cash
708,Statewide,January,2020,4.3%,2176109,4075962,500169011
709,Alameda,January,2020,3.1%,65302,112881,13678807
710,Alpine,January,2020,4.6%,77,126,14377
711,Amador,January,2020,4.7%,1496,2588,305508
712,Butte,January,2020,5.6%,17028,28648,3503997
...,...,...,...,...,...,...,...
1529,Tulare,February,2021,11.4%,45068,101481,17778143
1530,Tuolumne,February,2021,8.7%,3170,4907,786603
1531,Ventura,February,2021,6.8%,36565,67207,11082712
1532,Yolo,February,2021,6.8%,0,0,3572432


In [3]:
#CA County Data (Jackson)
#Data cleanup: Separate date into 3 columns, MONTH, DATE, YEAR. Isolate counties to CA. Drop FIPS column. 
all_counties = pd.read_csv(all_counties_path,index_col=0, parse_dates=True)

#Drop FIPS column
all_counties_df = pd.DataFrame(all_counties).drop(["fips"], axis=1)

#Isolate counties to CA 
CA_counties_df = all_counties_df[all_counties_df["state"] == "California"]
#CA_counties_df.head()

In [4]:
### ** MARK FOR REMOVAL --A value is trying to be set on a copy of a slice from a DataFrame ** ###
#Separate date into three columns and rename (Jackson)

#CA_counties_df.reset_index(inplace=True)
#CA_counties_df["Day"] = CA_counties_df["date"].dt.day
#CA_counties_df["Month"] = CA_counties_df["date"].dt.month
#CA_counties_df["Year"] = CA_counties_df["date"].dt.year
#CA_counties_df.rename(columns = {"date":"Date",
#                                "county":"County",
#                                "state":"State",
#                                "cases":"Cases",
#                                "deaths":"Deaths"}, inplace=True)

### ** MARK FOR REMOVAL ABOVE -----------------------------------------------** ###

CA_counties_df.reset_index(inplace=True)
CA_counties_df = CA_counties_df.astype({'date': str})
CA_counties_df[["year", "month", "day"]] = CA_counties_df["date"].str.split("-", expand = True)
CA_counties_df = CA_counties_df.astype({'deaths': int, 'year': int, 'month': int, 'day': int})
CA_counties_df.pop('date')  # Deleted original date column
CA_counties_df.head()
#print("\nNew DataFrame:")
#print(CA_counties_df)
#CA_counties_df.dtypes

,county,state,cases,deaths,year,month,day
0,Orange,California,1,0,2020,1,25
1,Los Angeles,California,1,0,2020,1,26
2,Orange,California,1,0,2020,1,26
3,Los Angeles,California,1,0,2020,1,27
4,Orange,California,1,0,2020,1,27


In [5]:

#orange_county = pd.DataFrame(CA_counties_df).groupby()

In [6]:
CA_counties_df.to_csv("Resources/CA_County_Data.csv")

In [7]:
# Filter to show necessary columns adn rename columns to use initial Caps.
census_data = pd.read_csv(census_data_path)
census_data.rename(columns = {'STNAME' : 'State', 'CTYNAME' : 'County', 
                              'POPESTIMATE2019' : 'Population'}, inplace = True)
census_df = pd.DataFrame(census_data)
census_df.drop(census_df.columns[[0, 1, 2, 3, 4, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 
                                  22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 
                                  39, 40, 41, 42, 43, 44, 45, 46, 47]], axis = 1, inplace = True)

# filter rows in state that equal California and required columns (Siddharth)
census_ca = census_df.loc[census_df['State'] == 'California']
census_filtered = census_ca[['State', 'County', 'Population', 'DEATHS2019']]
census_filtered.head()

,State,County,Population,DEATHS2019
190,California,California,39512223,282520
191,California,Alameda County,1671329,10944
192,California,Alpine County,1129,5
193,California,Amador County,39752,427
194,California,Butte County,219186,2527


In [8]:

# Combine CalFresh, census and counties data into a single dataset (JC)


# Create pd.DataFrame for population density by county. Timeframe 1-2020 to 3-2021

# Create pd.DataFrame for CalFresh recipients by county. Timeframe 1-2020 to 3-2021

# Create pd.DataFrame for total Covid cases by county. Timeframe 1-2020 to 3-2021
# Display the data table for preview


# Summary Statistics

## Line Chart 

In [9]:
# Show CA state covid cases over months using line plot. Years on x-axis and population on y-axis
# Timeframe: Jan 2020 to March 2021


In [10]:
# Graph trends amoungst various counties on a single plot. Pick 5, at least one urban area
# Timeframe: Jan 2020 to March 2021


In [11]:
# Correlation Rise of Covid and CalFresh (maybe unemployment) vs number of people receiving CalFresh aid.

### Bar Chart

In [12]:
# Number of Covid cases in all counties. Sort decending

In [13]:
# Number of Covid cases in top 10 counties. Sort decending